In [80]:
import json
import numpy as np
import pandas as pd
import requests

# Streaming 데이터
재생한 음악

* 'msPlayed' 컬럼은 그 음악을 여태까지 총 들은 시간이 아니라, 한번의 스트리밍에 얼마나 길게 들었는지를 의미함.
* 즉 '한곡재생'으로 해놓고 오래 두면 msPlayed가 높게 계산되는 형식.

## df_stream 생성

In [143]:
df1 = pd.read_json('C:/Users/n/Downloads/MyData/StreamingHistory0.json')
df1

,endTime,artistName,trackName,msPlayed
0,2021-07-15 14:46,Son Dam Bi,queen,16695
1,2021-07-16 07:15,Kim Areum,Aqua (feat. Muzie),299500
2,2021-07-16 07:16,D’allant,NEPTUNE,41056
3,2021-07-16 07:18,Hoody,When My Loneliness Calls You,129386
4,2021-07-16 07:20,Rainbow note,Morning,896
...,...,...,...,...
9995,2022-04-23 09:47,((( O ))),Shuh Shuh,62740
9996,2022-04-23 09:48,((( O ))),Une Verse,41447
9997,2022-04-23 09:49,((( O ))),Diwata,34946
9998,2022-04-23 09:49,((( O ))),Come Home O'Shawn,36432


In [144]:
df2 = pd.read_json('C:/Users/n/Downloads/MyData/StreamingHistory1.json')
df2

,endTime,artistName,trackName,msPlayed
0,2022-04-23 10:28,((( O ))),Je Suis Infini,183293
1,2022-04-23 10:29,((( O ))),Remember,24907
2,2022-04-23 10:31,((( O ))),Creation,143360
3,2022-04-23 10:33,((( O ))),Diwata,82477
4,2022-04-23 10:33,((( O ))),Creation,626
...,...,...,...,...
3938,2022-07-15 18:27,FKJ,Does It Exist,54853
3939,2022-07-16 01:17,FKJ,Stay A Child,150733
3940,2022-07-16 12:53,The Volunteers,Nicer,2986
3941,2022-07-16 12:58,The Volunteers,PINKTOP,239910


In [145]:
df_stream = pd.concat([df1, df2])
df_stream

,endTime,artistName,trackName,msPlayed
0,2021-07-15 14:46,Son Dam Bi,queen,16695
1,2021-07-16 07:15,Kim Areum,Aqua (feat. Muzie),299500
2,2021-07-16 07:16,D’allant,NEPTUNE,41056
3,2021-07-16 07:18,Hoody,When My Loneliness Calls You,129386
4,2021-07-16 07:20,Rainbow note,Morning,896
...,...,...,...,...
3938,2022-07-15 18:27,FKJ,Does It Exist,54853
3939,2022-07-16 01:17,FKJ,Stay A Child,150733
3940,2022-07-16 12:53,The Volunteers,Nicer,2986
3941,2022-07-16 12:58,The Volunteers,PINKTOP,239910


In [146]:
# 곡id 만들기
df_stream['id'] = df_stream['trackName'] + "-" + df_stream['artistName']
df_stream

,endTime,artistName,trackName,msPlayed,id
0,2021-07-15 14:46,Son Dam Bi,queen,16695,queen-Son Dam Bi
1,2021-07-16 07:15,Kim Areum,Aqua (feat. Muzie),299500,Aqua (feat. Muzie)-Kim Areum
2,2021-07-16 07:16,D’allant,NEPTUNE,41056,NEPTUNE-D’allant
3,2021-07-16 07:18,Hoody,When My Loneliness Calls You,129386,When My Loneliness Calls You-Hoody
4,2021-07-16 07:20,Rainbow note,Morning,896,Morning-Rainbow note
...,...,...,...,...,...
3938,2022-07-15 18:27,FKJ,Does It Exist,54853,Does It Exist-FKJ
3939,2022-07-16 01:17,FKJ,Stay A Child,150733,Stay A Child-FKJ
3940,2022-07-16 12:53,The Volunteers,Nicer,2986,Nicer-The Volunteers
3941,2022-07-16 12:58,The Volunteers,PINKTOP,239910,PINKTOP-The Volunteers


## df_stream EDA

In [60]:
# 결측치 확인
df_stream.isna().sum()

endTime        0
artistName     0
trackName      0
msPlayed       0
id             0
liked_track    0
dtype: int64

In [42]:
# 데이터 기간 확인
df_stream.endTime.min(), df_stream.endTime.max()

('2021-07-15 14:46', '2022-07-16 13:01')

In [57]:
# 곡과 가수의 가짓수 확인
# 중복되는 곡이 여러번 나오는 구조라 nunique 필요
df_stream.nunique() # 1336명의 아티스트, 4040개의 곡

endTime        11112
artistName      1336
trackName       3876
msPlayed        6855
id              4040
liked_track        2
dtype: int64

### 많이 들은 30곡
* 좋아해서 많이 들었던 곡보다 '공부할 때 듣기 좋은 잔잔한 곡' 혹은 '안무로 쓰인 노래라 연습 때 반복으로 틀어놨던 곡'이 대부분임
* 단순히 들은 시간(msPlayed)이 많다는 것만으로 취향을 분석할 수 없는 한계를 찾음

In [58]:
# 많이 들은 30곡
df_stream.groupby(['id'], as_index=False)['msPlayed'].sum().sort_values(by='msPlayed', ascending=False).reset_index(drop=True).head(30)

,id,msPlayed
0,If We're Being Honest-Novo Amor,22992535
1,"I Know (feat. EK, BOLA) [Prod. Neal]-MBA",22626210
2,Love Line (with Tinashe)-Shift K3Y,17106542
3,Tadow-Masego,16869274
4,attention-Omah Lay,16687063
5,Feelin Right-Malachiae,15097767
6,Love U Better-Victoria Monét,14864871
7,Miss You (with Major Lazer & Tory Lanez)-Cashm...,13928344
8,Let It-CL,13840100
9,altar-Kehlani,12368431


### 많이 들은 30 아티스트
* 위와 같은 경우로 많이 잡힌 아티스트가 두어개 있지만 대부분 '내가 생각하던 내 취향'과 더 일치하는 결과를 보여줬음

In [137]:
# 많이 들은 30 아티스트
df_stream.groupby(['artistName'], as_index=False)['msPlayed'].sum().sort_values(by='msPlayed', ascending=False).head(30).reset_index(drop=True)

,artistName,msPlayed
0,Yerin Baek,137918538
1,The 1975,85139210
2,Kehlani,40652129
3,The Volunteers,35082886
4,beabadoobee,32954265
5,Luli Lee,31958654
6,The Black Skirts,29975710
7,TRPP,28077420
8,Novo Amor,27820588
9,CL,24323048


### 많이 들은 아티스트별 최애곡
* 많이 들은 Top5 아티스트의 곡 중 뭘 가장 많이 들었는지 (msPlayed의 합)
* 많이 들은 Top5 아티스트의 곡 중 뭘 가장 자주 들었는지 (횟수)
* 다섯 아티스트 모두 둘의 결과가 동일했음

In [158]:
df_stream[(df_stream['artistName']=='Yerin Baek')].groupby(['id'], as_index=False)['msPlayed'].sum().sort_values(by='msPlayed', ascending=False).reset_index(drop=True).head(1)

,id,msPlayed
0,Antifreeze-Yerin Baek,10717179


In [154]:
df_stream[(df_stream['artistName']=='Yerin Baek')]['id'].value_counts().head(1)

Antifreeze-Yerin Baek    49
Name: id, dtype: int64

In [159]:
df_stream[(df_stream['artistName']=='The 1975')].groupby(['id'], as_index=False)['msPlayed'].sum().sort_values(by='msPlayed', ascending=False).reset_index(drop=True).head(1)

,id,msPlayed
0,Robbers-The 1975,4804546


In [160]:
df_stream[(df_stream['artistName']=='The 1975')]['id'].value_counts().head(1)

Robbers-The 1975    22
Name: id, dtype: int64

In [161]:
df_stream[(df_stream['artistName']=='Kehlani')].groupby(['id'], as_index=False)['msPlayed'].sum().sort_values(by='msPlayed', ascending=False).reset_index(drop=True).head(1)

,id,msPlayed
0,altar-Kehlani,12368431


In [162]:
df_stream[(df_stream['artistName']=='Kehlani')]['id'].value_counts().head(1)

altar-Kehlani    56
Name: id, dtype: int64

In [163]:
df_stream[(df_stream['artistName']=='The Volunteers')].groupby(['id'], as_index=False)['msPlayed'].sum().sort_values(by='msPlayed', ascending=False).reset_index(drop=True).head(1)

,id,msPlayed
0,Summer-The Volunteers,4696787


In [164]:
df_stream[(df_stream['artistName']=='The Volunteers')]['id'].value_counts().head(1)

Summer-The Volunteers    21
Name: id, dtype: int64

In [165]:
df_stream[(df_stream['artistName']=='beabadoobee')].groupby(['id'], as_index=False)['msPlayed'].sum().sort_values(by='msPlayed', ascending=False).reset_index(drop=True).head(1)

,id,msPlayed
0,Last Day On Earth-beabadoobee,2724549


In [167]:
df_stream[(df_stream['artistName']=='beabadoobee')]['id'].value_counts().head(1)

Last Day On Earth-beabadoobee    16
Name: id, dtype: int64

### 여러번 들었던 곡
* 들은 시간(msPlayed)이 많은 게 아니라 들은 횟수(count)가 많은 음악
* 들은 시간이 많은 음악과 비슷한 결과임

In [121]:
df_stream['id'].value_counts().head(30)

I Know (feat. EK, BOLA) [Prod. Neal]-MBA                 123
Love Line (with Tinashe)-Shift K3Y                       110
If We're Being Honest-Novo Amor                           98
attention-Omah Lay                                        93
Feelin Right-Malachiae                                    90
Let It-CL                                                 80
Miss You (with Major Lazer & Tory Lanez)-Cashmere Cat     78
Love U Better-Victoria Monét                              73
Feelin U (feat. Demarco, Doctor, Ras Kwame)-KickRaux      67
One Last Time-Ariana Grande                               64
E.T.A.-Justin Bieber                                      62
Tadow-Masego                                              60
Ylang Ylang-FKJ                                           58
Stupid Love-Lady Gaga                                     56
altar-Kehlani                                             56
BTBT (feat. DeVita)-B.I                                   52
Next Level - IMLAY Remix

### 들은 곡 중에 like 누른 곡
* 4040개의 df_stream 곡 중에 좋아요 누른 곡은 1055곡이었음
* 평균적으로 듣는 곡의 26%를 맘에 들어함

In [119]:
 # 1055곡
stream_and_like = df_stream[(df_stream['liked_track'] == True)]
stream_and_like.groupby(['id']).count()

,endTime,artistName,trackName,msPlayed,liked_track
id,,,,,
(what i wish just one person would say to me)-LANY,1,1,1,1,1
0310-Yerin Baek,25,25,25,25,25
0415 - KIRARA Remix-Yerin Baek,6,6,6,6,6
0415-Yerin Baek,14,14,14,14,14
0X1=LOVESONG (I Know I Love You) feat. Seori (Emocore Mix)-TOMORROW X TOGETHER,2,2,2,2,2
...,...,...,...,...,...
인공눈물 Our Lust-Bye Bye Badman,1,1,1,1,1
지켜줄게-Yerin Baek,8,8,8,8,8
집 Home (feat. 유미 Youme)-House Rulez,2,2,2,2,2


### 많이 들었지만 like 안한 곡
* 6곡은 안무곡이라 자주 들었음
* Yerin Baek의 4곡은 예외적으로 안무곡이 아니었음, 앨범 전체재생 때문인 것으로 추정 

In [134]:
stream_and_not_like = df_stream[(df_stream['liked_track'] == False)]
stream_and_not_like.groupby(['id']).sum().sort_values(by='msPlayed', ascending=False).head(10)

,msPlayed,liked_track
id,,
Rewrite The Stars-Zac Efron,5762827,0
Trust - Remix-Buju Banton,5727784,0
love nwantiti (ah ah ah)-CKay,5703224,0
Jolie nana-Aya Nakamura,3534351,0
Pose-Rihanna,3273593,0
lovelovelove-Yerin Baek,2460375,0
London-Yerin Baek,2420292,0
Run It Up (feat. Offset & Moneybagg Yo)-Lil Tjay,2373334,0
Not a girl-Yerin Baek,2068776,0


### 여러번 들었지만 like 안한 곡
*  여러번(횟수 개념) 들었지만 like 안한 곡
* 안무곡이었거나 들은 횟수가 그리 많지 않음

In [136]:
stream_and_not_like = df_stream[(df_stream['liked_track'] == False)]
stream_and_not_like.groupby(['id']).count().sort_values(by='endTime', ascending=False).head(10)

,endTime,artistName,trackName,msPlayed,liked_track
id,,,,,
love nwantiti (ah ah ah)-CKay,49,49,49,49,49
Risk-FKJ,49,49,49,49,49
Trust - Remix-Buju Banton,38,38,38,38,38
Rewrite The Stars-Zac Efron,34,34,34,34,34
Pose-Rihanna,30,30,30,30,30
Jolie nana-Aya Nakamura,28,28,28,28,28
Brother-FKJ,28,28,28,28,28
London-Yerin Baek,15,15,15,15,15
Intro-Yerin Baek,14,14,14,14,14


### 분기별 최다 스트리밍곡
* 대부분 '많이 들은 30 아티스트'에 포함되지 않은 점이 흥미로움
* 해외 팝송(신곡) 위주
* '은은하게 항상 깔고가는 취향'에는 정해진 아티스트가 있지만, 특정 시기에 꽂힌 곡은 평소의 취향과는 다른 결의 개별곡임을 알 수 있음

In [125]:
# 2021-07-15 ~ 2021-09-30
quarter1 = df_stream[('2021-07-15' <= df_stream['endTime']) & (df_stream['endTime'] < '2021-10-01')]
quarter1['id'].value_counts().head(5)

If We're Being Honest-Novo Amor                 91
STAY (with Justin Bieber)-The Kid LAROI         36
Antifreeze-Yerin Baek                           33
Us2-YULTRON                                     27
Put It All on Me (feat. Ella Mai)-Ed Sheeran    24
Name: id, dtype: int64

In [126]:
# 2021-10-01 ~ 2021-12-31
quarter2 = df_stream[('2021-10-01' <= df_stream['endTime']) & (df_stream['endTime'] < '2022-01-01')]
quarter2['id'].value_counts().head(5)

Let It-CL                                               71
Feelin U (feat. Demarco, Doctor, Ras Kwame)-KickRaux    61
Risk-FKJ                                                48
Love U Better-Victoria Monét                            45
40%-Aya Nakamura                                        42
Name: id, dtype: int64

In [127]:
# 2022-01-01 ~ 2022-03-31
quarter3 = df_stream[('2022-01-01' <= df_stream['endTime']) & (df_stream['endTime'] < '2022-04-01')]
quarter3['id'].value_counts().head(5)

Love Line (with Tinashe)-Shift K3Y                       100
Miss You (with Major Lazer & Tory Lanez)-Cashmere Cat     74
Stupid Love-Lady Gaga                                     47
Bonnie & Clyde - Korean Version-DeVita                    43
altar-Kehlani                                             43
Name: id, dtype: int64

In [128]:
# 2022-04-01 ~ 2022-06-30
quarter4 = df_stream[('2021-04-01' <= df_stream['endTime']) & (df_stream['endTime'] < '2022-07-01')]
quarter4['id'].value_counts().head(5)

I Know (feat. EK, BOLA) [Prod. Neal]-MBA    122
Love Line (with Tinashe)-Shift K3Y          108
If We're Being Honest-Novo Amor              98
attention-Omah Lay                           93
Feelin Right-Malachiae                       89
Name: id, dtype: int64

# Library 관련 데이터

## tracks 데이터
좋아요 표시한 곡 1115개

In [44]:
track = pd.read_json('C:/Users/n/Downloads/MyData/Library_tracks.json', orient='split')
track

,artist,album,track,uri
0,Phoebe Bridgers,Stranger in the Alps,Scott Street,spotify:track:6Uwi2Qk3H7fM4b4W4ExrAp
1,ROSÉ,R,Gone,spotify:track:2ayIgfvWo3SfYP2pVOr4pC
2,The 1975,The 1975,The City,spotify:track:2VeL9b7uEZ35q1B25QPehz
3,Swim Deep,On the Floor,On the Floor,spotify:track:0qmrZ3CCRrdh0gxDKjmD6G
4,Fromm,Milan Blue,Milan Blue,spotify:track:2B64LQET1AVf85fVJDTOXD
...,...,...,...,...
1110,Novo Amor,If We're Being Honest,If We're Being Honest,spotify:track:6CfAtjogau9v9kmtZVrNG8
1111,Charly Black,You're Perfect,You're Perfect,spotify:track:4iSWQ5Eadn5tZvvG9XNzYN
1112,2NE1,2NE1 1st Mini Album,Stay Together,spotify:track:3dbDqyQQz5MUhcrQEJOHFP
1113,JO YURI,GLASSY,Express Moon,spotify:track:3n2FfnO1tjMxyWHofqKX7S


In [45]:
# 곡id 만들기
track['id'] = track['track'] + "-" + track['artist']
track

,artist,album,track,uri,id
0,Phoebe Bridgers,Stranger in the Alps,Scott Street,spotify:track:6Uwi2Qk3H7fM4b4W4ExrAp,Scott Street-Phoebe Bridgers
1,ROSÉ,R,Gone,spotify:track:2ayIgfvWo3SfYP2pVOr4pC,Gone-ROSÉ
2,The 1975,The 1975,The City,spotify:track:2VeL9b7uEZ35q1B25QPehz,The City-The 1975
3,Swim Deep,On the Floor,On the Floor,spotify:track:0qmrZ3CCRrdh0gxDKjmD6G,On the Floor-Swim Deep
4,Fromm,Milan Blue,Milan Blue,spotify:track:2B64LQET1AVf85fVJDTOXD,Milan Blue-Fromm
...,...,...,...,...,...
1110,Novo Amor,If We're Being Honest,If We're Being Honest,spotify:track:6CfAtjogau9v9kmtZVrNG8,If We're Being Honest-Novo Amor
1111,Charly Black,You're Perfect,You're Perfect,spotify:track:4iSWQ5Eadn5tZvvG9XNzYN,You're Perfect-Charly Black
1112,2NE1,2NE1 1st Mini Album,Stay Together,spotify:track:3dbDqyQQz5MUhcrQEJOHFP,Stay Together-2NE1
1113,JO YURI,GLASSY,Express Moon,spotify:track:3n2FfnO1tjMxyWHofqKX7S,Express Moon-JO YURI


In [46]:
# uri컬럼 문자열 정리
track['uri'] = track["uri"].str.split(":", expand = True)[2]
track

,artist,album,track,uri,id
0,Phoebe Bridgers,Stranger in the Alps,Scott Street,6Uwi2Qk3H7fM4b4W4ExrAp,Scott Street-Phoebe Bridgers
1,ROSÉ,R,Gone,2ayIgfvWo3SfYP2pVOr4pC,Gone-ROSÉ
2,The 1975,The 1975,The City,2VeL9b7uEZ35q1B25QPehz,The City-The 1975
3,Swim Deep,On the Floor,On the Floor,0qmrZ3CCRrdh0gxDKjmD6G,On the Floor-Swim Deep
4,Fromm,Milan Blue,Milan Blue,2B64LQET1AVf85fVJDTOXD,Milan Blue-Fromm
...,...,...,...,...,...
1110,Novo Amor,If We're Being Honest,If We're Being Honest,6CfAtjogau9v9kmtZVrNG8,If We're Being Honest-Novo Amor
1111,Charly Black,You're Perfect,You're Perfect,4iSWQ5Eadn5tZvvG9XNzYN,You're Perfect-Charly Black
1112,2NE1,2NE1 1st Mini Album,Stay Together,3dbDqyQQz5MUhcrQEJOHFP,Stay Together-2NE1
1113,JO YURI,GLASSY,Express Moon,3n2FfnO1tjMxyWHofqKX7S,Express Moon-JO YURI


### tracks 정보 df_stream에 추가
* df_stream 이제 완성!

In [147]:
# df_stream에 좋아요 표시한 곡인지 T/F 컬럼 추가
df_stream['liked_track'] = np.where(df_stream['id'].isin(track['id'].tolist()),True,False)
df_stream

,endTime,artistName,trackName,msPlayed,id,liked_track
0,2021-07-15 14:46,Son Dam Bi,queen,16695,queen-Son Dam Bi,False
1,2021-07-16 07:15,Kim Areum,Aqua (feat. Muzie),299500,Aqua (feat. Muzie)-Kim Areum,False
2,2021-07-16 07:16,D’allant,NEPTUNE,41056,NEPTUNE-D’allant,False
3,2021-07-16 07:18,Hoody,When My Loneliness Calls You,129386,When My Loneliness Calls You-Hoody,False
4,2021-07-16 07:20,Rainbow note,Morning,896,Morning-Rainbow note,False
...,...,...,...,...,...,...
3938,2022-07-15 18:27,FKJ,Does It Exist,54853,Does It Exist-FKJ,False
3939,2022-07-16 01:17,FKJ,Stay A Child,150733,Stay A Child-FKJ,False
3940,2022-07-16 12:53,The Volunteers,Nicer,2986,Nicer-The Volunteers,True
3941,2022-07-16 12:58,The Volunteers,PINKTOP,239910,PINKTOP-The Volunteers,True


## album 데이터
좋아요 표시한 앨범

In [49]:
album = pd.read_json('C:/Users/n/Downloads/MyData/Library_album.json', orient='split')
album

,artist,album,uri
0,Lady Gaga,Chromatica,spotify:album:05c49JgPmL4Uz2ZeqRx5SP
1,FKJ,V I N C E N T,spotify:album:5BWHI6HIlttptflig3odJ7
2,Vampire Weekend,Modern Vampires of the City,spotify:album:1GXMNFfoHF4sN7lG8gZq1j
3,Mystery Jets,A Billion Heartbeats,spotify:album:7DF4LKdCncixDcFN9UAbVp
4,RADWIMPS,Your Name.,spotify:album:4qApTp9557qYZzRLEih4uP
5,LANY,mama's boy,spotify:album:6ZsCg58PZj6jFALfiB5ZtX
6,The 1975,Rdio Sessions,spotify:album:4bNfEWA3Ib15SIqEy3febD
7,Yerin Baek,Every letter I sent you.,spotify:album:20hW2P3VSNJ1A7MwjIJ0Up
8,TRPP,TRPP,spotify:album:2BacozGtXMOerDsVq5lJkd
9,The Volunteers,The Volunteers,spotify:album:0hwXFwuvVfZwn6asTargTD


In [51]:
# track 데이터에 좋아요 누른 앨범인지 T/F컬럼 추가
track['liked_album'] = np.where(track['album'].isin(album['album'].tolist()),True,False)
track

,artist,album,track,uri,id,liked_album
0,Phoebe Bridgers,Stranger in the Alps,Scott Street,6Uwi2Qk3H7fM4b4W4ExrAp,Scott Street-Phoebe Bridgers,False
1,ROSÉ,R,Gone,2ayIgfvWo3SfYP2pVOr4pC,Gone-ROSÉ,False
2,The 1975,The 1975,The City,2VeL9b7uEZ35q1B25QPehz,The City-The 1975,True
3,Swim Deep,On the Floor,On the Floor,0qmrZ3CCRrdh0gxDKjmD6G,On the Floor-Swim Deep,False
4,Fromm,Milan Blue,Milan Blue,2B64LQET1AVf85fVJDTOXD,Milan Blue-Fromm,False
...,...,...,...,...,...,...
1110,Novo Amor,If We're Being Honest,If We're Being Honest,6CfAtjogau9v9kmtZVrNG8,If We're Being Honest-Novo Amor,False
1111,Charly Black,You're Perfect,You're Perfect,4iSWQ5Eadn5tZvvG9XNzYN,You're Perfect-Charly Black,False
1112,2NE1,2NE1 1st Mini Album,Stay Together,3dbDqyQQz5MUhcrQEJOHFP,Stay Together-2NE1,False
1113,JO YURI,GLASSY,Express Moon,3n2FfnO1tjMxyWHofqKX7S,Express Moon-JO YURI,False


## artist 데이터
좋아요 표시한 아티스트

In [52]:
artist = pd.read_json('C:/Users/n/Downloads/MyData/Library_artist.json', orient='split')
artist

,name,uri
0,(G)I-DLE,spotify:artist:2AfmfGFbe0A0WsTYm0SDTx
1,Agust D,spotify:artist:5RmQ8k4l3HZ8JoPb4mNsML
2,Alvvays,spotify:artist:3kzwYV3OCB010YfXMF0Avt
3,BLACKPINK,spotify:artist:41MozSoPIsD1dJM0CLPjZF
4,Beach Fossils,spotify:artist:1bwUhKRmEkOZ1wuTnV9XjC
5,Bye Bye Badman,spotify:artist:14stFRB2PwF6QdKNHDdMnh
6,CNEMA,spotify:artist:1rH9ZFWkeghhAktUmri9O6
7,COIN,spotify:artist:0ZxZlO7oWCSYMXhehpyMvE
8,FKJ,spotify:artist:2FwDTncULUnmANIh7qKa5z
9,Joji,spotify:artist:3MZsBdqDrRTJihTHQrO6Dq


In [54]:
# track 데이터에, 좋아요 누른 아티스트인지 T/F컬럼 추가
track['liked_artist'] = np.where(track['artist'].isin(artist['name'].tolist()),True,False)
track

,artist,album,track,uri,id,liked_album,liked_artist
0,Phoebe Bridgers,Stranger in the Alps,Scott Street,6Uwi2Qk3H7fM4b4W4ExrAp,Scott Street-Phoebe Bridgers,False,False
1,ROSÉ,R,Gone,2ayIgfvWo3SfYP2pVOr4pC,Gone-ROSÉ,False,False
2,The 1975,The 1975,The City,2VeL9b7uEZ35q1B25QPehz,The City-The 1975,True,True
3,Swim Deep,On the Floor,On the Floor,0qmrZ3CCRrdh0gxDKjmD6G,On the Floor-Swim Deep,False,True
4,Fromm,Milan Blue,Milan Blue,2B64LQET1AVf85fVJDTOXD,Milan Blue-Fromm,False,False
...,...,...,...,...,...,...,...
1110,Novo Amor,If We're Being Honest,If We're Being Honest,6CfAtjogau9v9kmtZVrNG8,If We're Being Honest-Novo Amor,False,False
1111,Charly Black,You're Perfect,You're Perfect,4iSWQ5Eadn5tZvvG9XNzYN,You're Perfect-Charly Black,False,False
1112,2NE1,2NE1 1st Mini Album,Stay Together,3dbDqyQQz5MUhcrQEJOHFP,Stay Together-2NE1,False,False
1113,JO YURI,GLASSY,Express Moon,3n2FfnO1tjMxyWHofqKX7S,Express Moon-JO YURI,False,False


## df_library 생성
* df_library는 df_stream과 달리 한곡당 한행씩만 차지함

In [56]:
df_library = track.copy()
df_library

,artist,album,track,uri,id,liked_album,liked_artist
0,Phoebe Bridgers,Stranger in the Alps,Scott Street,6Uwi2Qk3H7fM4b4W4ExrAp,Scott Street-Phoebe Bridgers,False,False
1,ROSÉ,R,Gone,2ayIgfvWo3SfYP2pVOr4pC,Gone-ROSÉ,False,False
2,The 1975,The 1975,The City,2VeL9b7uEZ35q1B25QPehz,The City-The 1975,True,True
3,Swim Deep,On the Floor,On the Floor,0qmrZ3CCRrdh0gxDKjmD6G,On the Floor-Swim Deep,False,True
4,Fromm,Milan Blue,Milan Blue,2B64LQET1AVf85fVJDTOXD,Milan Blue-Fromm,False,False
...,...,...,...,...,...,...,...
1110,Novo Amor,If We're Being Honest,If We're Being Honest,6CfAtjogau9v9kmtZVrNG8,If We're Being Honest-Novo Amor,False,False
1111,Charly Black,You're Perfect,You're Perfect,4iSWQ5Eadn5tZvvG9XNzYN,You're Perfect-Charly Black,False,False
1112,2NE1,2NE1 1st Mini Album,Stay Together,3dbDqyQQz5MUhcrQEJOHFP,Stay Together-2NE1,False,False
1113,JO YURI,GLASSY,Express Moon,3n2FfnO1tjMxyWHofqKX7S,Express Moon-JO YURI,False,False


## df_library EDA

### like한 곡이 많은 30 아티스트
* '많이 들은 30 아티스트'와 비교하면,
* 30곡 이상 like한 경우 탄탄한 팬심이 있다고 볼 수 있지만,
* LANY, Bye Bye Badman는 like는 많이 한거에 비해 자주 듣진 않았다

In [101]:
df_library['artist'].value_counts().head(30)

The 1975               56
The Black Skirts       40
beabadoobee            33
Yerin Baek             30
LANY                   24
Bye Bye Badman         21
(G)I-DLE               19
TRPP                   18
BLACKPINK              15
Oasis                  13
Luli Lee               13
The Volunteers         12
Swim Deep              12
Alvvays                12
TOMORROW X TOGETHER    11
Nerd Connection        11
Mystery Jets           10
Kehlani                10
Alina Baraz            10
Dermot Kennedy         10
Minsu                  10
G-DRAGON               10
Porter Robinson         9
The War On Drugs        9
Harry Styles            9
Lady Gaga               9
FKJ                     9
TAEYEON                 9
BIGBANG                 8
IU                      8
Name: artist, dtype: int64

### like한 곡이 많은 앨범

In [174]:
df_library.groupby(['album']).count().sort_values(by='id', ascending=False).head(10)

,artist,track,uri,id,liked_album,liked_artist
album,,,,,,
The 1975,24,24,24,24,24,24
THIRSTY,18,18,18,18,18,18
mama's boy,17,17,17,17,17,17
"I like it when you sleep, for you are so beautiful yet so unaware of it",11,11,11,11,11,11
TEAM BABY,10,10,10,10,10,10
The Volunteers,10,10,10,10,10,10
tellusboutyourself,10,10,10,10,10,10
Notes On A Conditional Form,10,10,10,10,10,10
AUTHENTIC,9,9,9,9,9,9


# Spotify API로 genre정보 끌어오기
* uri를 갖고 있는 df_library는 스포티파이API를 이용해 장르 정보를 끌고 올 수 있음
* 참고자료 https://towardsdatascience.com/visualizing-spotify-data-with-python-tableau-687f2f528cdd

In [86]:
# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = '----'
CLIENT_SECRET = '----'

In [87]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [88]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [89]:
# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['uri'].to_list()

# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

In [90]:
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

In [95]:
df_genre

,track_uri,artist_uri,genres
0,6Uwi2Qk3H7fM4b4W4ExrAp,1r1uxoy19fzMxunt3ONAkG,"[indie pop, la indie]"
1,2ayIgfvWo3SfYP2pVOr4pC,3eVa5w3URK5duf6eyVDbu9,[k-pop]
2,2VeL9b7uEZ35q1B25QPehz,3mIj9lX2MWuHmhNCA7LSCW,"[modern alternative rock, modern rock, pop, rock]"
3,0qmrZ3CCRrdh0gxDKjmD6G,2XddLUPFBmTonCFy8uB3uc,"[birmingham indie, english indie rock, shiver ..."
4,2B64LQET1AVf85fVJDTOXD,4ht1mwP4ouSu79jOOktCY3,"[k-indie, korean ost]"
...,...,...,...
1110,6CfAtjogau9v9kmtZVrNG8,0rZp7G3gIH6WkyeXbrZnGi,"[ambient folk, indie folk, pop]"
1111,4iSWQ5Eadn5tZvvG9XNzYN,5sK8BsvyDl4TFA6KaBf8or,"[dancehall, jamaican dancehall]"
1112,3dbDqyQQz5MUhcrQEJOHFP,1l0mKo96Jh9HVYONcRl3Yp,"[dance pop, k-pop, k-pop girl group]"
1113,3n2FfnO1tjMxyWHofqKX7S,3LFFf4EpKn2krneZ9vozyz,"[k-pop, korean ost]"


In [93]:
# 분석하기 좋게 한 키워드씩 분리
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded

,track_uri,artist_uri,genres
0,6Uwi2Qk3H7fM4b4W4ExrAp,1r1uxoy19fzMxunt3ONAkG,indie pop
0,6Uwi2Qk3H7fM4b4W4ExrAp,1r1uxoy19fzMxunt3ONAkG,la indie
1,2ayIgfvWo3SfYP2pVOr4pC,3eVa5w3URK5duf6eyVDbu9,k-pop
2,2VeL9b7uEZ35q1B25QPehz,3mIj9lX2MWuHmhNCA7LSCW,modern alternative rock
2,2VeL9b7uEZ35q1B25QPehz,3mIj9lX2MWuHmhNCA7LSCW,modern rock
...,...,...,...
1112,3dbDqyQQz5MUhcrQEJOHFP,1l0mKo96Jh9HVYONcRl3Yp,k-pop
1112,3dbDqyQQz5MUhcrQEJOHFP,1l0mKo96Jh9HVYONcRl3Yp,k-pop girl group
1113,3n2FfnO1tjMxyWHofqKX7S,3LFFf4EpKn2krneZ9vozyz,k-pop
1113,3n2FfnO1tjMxyWHofqKX7S,3LFFf4EpKn2krneZ9vozyz,korean ost


## df_genre EDA

### 많이 like한 30 장르 키워드

In [106]:
df_genre['genres'].value_counts().head(30)

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[k-indie]                                                                                                                    93
[]                                                                                                                           84
[modern alternative rock, modern rock, pop, rock]                                                                            56
[k-pop]                                                                                                                      51
[k-pop, k-pop girl group]                                                                                                    50
[k-indie, k-rock, korean dream pop, korean indie rock]                                                                       40
[bedroom pop, bubblegrunge, indie pop, pop]                                                                                  33
[k-pop, korean pop]                                                                                     

In [105]:
df_genre_expanded['genres'].value_counts().head(30)

pop                        270
k-indie                    224
k-pop                      187
modern rock                109
rock                        95
korean indie rock           93
k-rock                      84
indie pop                   78
dance pop                   72
modern alternative rock     63
k-pop girl group            63
indie rock                  59
korean dream pop            57
uk pop                      50
korean pop                  46
korean city pop             41
r&b                         40
bedroom pop                 39
bubblegrunge                34
pop rap                     32
permanent wave              32
k-rap                       26
electropop                  25
shoegaze                    25
la pop                      24
art pop                     24
english indie rock          23
k-pop boy group             23
alternative rock            22
korean r&b                  20
Name: genres, dtype: int64

# csv로 저장
* 세개의 데이터를 csv로 저장해 Tableu로 시각화할 예정
* df_stream은 시계열 분석을 주로 할 수 있고
* df_library는 장르와 연관된 분석을 주로 할 수 있을 것으로 보임

In [ ]:
df_stream.to_csv('C:/Users/n/Downloads/df_stream.csv')
df_library.to_csv('C:/Users/n/Downloads/df_library.csv')
df_genre_expanded.to_csv('C:/Users/n/Downloads/df_genre_expanded.csv')

print('done')